In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/December_projects/cyberbully/processed_1L_data.csv')

df.head()

,text,label
0,@Seveerity I guess only things she got is that...,0
1,@Stravon_ @Christcucks2 You said the nigger wo...,0
2,@TheVillageWareh @matinyarare @glen_mogau This...,0
3,@KBthabuddhist Nigga really singin man,0
4,@ImmuneHack @ScottAdamsSays I would take out t...,0


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load pre-trained DistilBERT model for multiclass classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',  num_labels=len(df_filtered['label'].unique()))  # Adjust num_labels as per your classes
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#model = DistilBertForSequenceClassification.from_pretrained('/content/results/checkpoint-1788')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
from sklearn.model_selection import train_test_split
# Split the data into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset
# Converting the DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

def preprocess_function(examples):
    examples['labels'] = examples['label']
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

from sklearn.metrics import accuracy_score
# Defining compute_metrics function to calculate accuracy
def compute_metrics(p):
    preds = p.predictions.argmax(-1)  # Get predicted class (argmax of logits)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)  # Compute accuracy
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir='./results',              # Where to save the model and checkpoints
    evaluation_strategy="epoch",         # Evaluate every epoch
    save_strategy="epoch",              # Save checkpoint every epoch
    logging_dir='./logs',                # Directory for logs
    num_train_epochs=3,                  # Number of epochs
    per_device_train_batch_size=64,       # Batch size for training
    per_device_eval_batch_size=64,        # Batch size for evaluation
    logging_steps=10,                    # Log every 10 steps
    save_total_limit=3,                  # Keep only the last 3 checkpoints
    load_best_model_at_end=True,         # Load the best model at the end of training
    report_to=[],  # Disable W&B logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # compute_metrics function
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.029100,0.021092,0.995583
2,0.000900,0.016549,0.996333
3,0.002400,0.016935,0.996333


TrainOutput(global_step=2250, training_loss=0.027451882236430216, metrics={'train_runtime': 6892.9625, 'train_samples_per_second': 20.891, 'train_steps_per_second': 0.326, 'total_flos': 1.9075985768448e+16, 'train_loss': 0.027451882236430216, 'epoch': 3.0})

In [ ]:
# Saving model
trainer.save_model("/content/drive/MyDrive/December_projects/cyberbully")

In [ ]:
# real time testing

In [ ]:
# Load the saved model
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

loaded_model = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/December_projects/cyberbully")
loaded_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased") # Ensure you use the same tokenizer as during training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import pickle
# Load the saved model
loaded_model = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/December_projects/cyberbully")
loaded_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased") # Ensure you use the same tokenizer as during training

# Load the label mapping
with open('/content/drive/MyDrive/December_projects/cyberbully/label_mapping.pkl', 'rb') as f:
    label_mapping = pickle.load(f)

In [ ]:
label_mapping

{'ethnicity/race': 0,
 'gender/sexual': 1,
 'not_cyberbullying': 2,
 'religion': 3}

In [ ]:
# random record selection

import random

# can Filter for records according to label
label_0_df = df[df['label'] == 'ethnicity/race']

# Selecting a random record
random_record = label_0_df.sample(n=1)

# Convert the 'text' column of the random record to a string
random_text = random_record['text'].iloc[0]

random_text

'@Ry_2The_Ann That’s a young nigga saying lol'

In [ ]:
# real time prediction

import torch

def predict_cyberbullying(text, model, tokenizer, label_mapping):
    # Tokenize the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)

    # Get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Get the original label
    for original_label, encoded_label in label_mapping.items():
      if encoded_label == predicted_class:
        predicted_label = original_label
        break

    return predicted_label, probabilities

In [ ]:
input_text = random_text

predicted_label, probabilities = predict_cyberbullying(input_text, loaded_model, loaded_tokenizer, label_mapping)

print(f"Predicted label: {predicted_label}")
print(f"Probabilities: {probabilities}")

Predicted label: ethnicity/race
Probabilities: tensor([[9.9959e-01, 2.7986e-04, 3.8377e-05, 9.1833e-05]])
